In [30]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_recall_fscore_support
from collections import Counter
from IPython.display import clear_output

from pyexplainer.pyexplainer_pyexplainer import PyExplainer
from pyexplainer.pyexplainer_pyexplainer import AutoSpearman

In [5]:
ranked_tp_features = open("ranked_feature_names/ranked(GS)_tp_200_combined.txt").read().split('\n')[0:-1]
#ranked_tp_features = open("mydataset/100_correctly_predicted_buggy_fetature_names.txt").read().split('\n')[0:-1]
train_commit = open("mydataset/train_commit.txt").read().split('\n')[0:-1]
test_commit = open("mydataset/test_commit.txt").read().split('\n')[0:-1]
train_label = open("mydataset/train_label.txt").read().split('\n')[0:-1]
test_label = open("mydataset/test_label.txt").read().split('\n')[0:-1]
print(ranked_tp_features[0:10])
print(len(train_commit), len(test_commit), len(train_label), len(test_label))

['la', 'asawr', 'oexp', 'nf', 'age', 'osawr', 'rtime', 'hcmt', 'arexp', 'ent']
1709 734 1709 734


In [6]:
def findCorrectBuggy(yTest, yPredict):
    #length of these arrays will determine the number of correct buggy and non buggy commits
    CorrectBuggyCommits = []
    CorrectNonBuggyCommits = []
    
    #CorrectbuggyCount = 0
    #CorrectNonbuggyCount = 0
    for index in range(len(yTest)):
        if(yTest[index] == 1 and yPredict[index] == 1):
            #CorrectbuggyCount += 1
            CorrectBuggyCommits.append(test_commit[index])
        elif(yTest[index] == 0 and yPredict[index] == 0):
            #CorrectNonbuggyCount += 1
            CorrectNonBuggyCommits.append(test_commit[index])
            
    return CorrectBuggyCommits, CorrectNonBuggyCommits

In [9]:
# [['commit_id']+ranked_tp_features[0:]]
file_path = "mydataset/"
#read three data files and merge into one dataframe 'inner_join' by 'commit_id'
df_tp = pd.merge(pd.read_csv(file_path+ "qtbase_tp_dataset.csv"), #TP: Token Pattern Features
                 pd.read_csv(file_path+ "qt_metrics.csv"),
                 on='commit_id')[['commit_id']+ranked_tp_features[0:]]
#print(df_tp.columns)

df_tp.replace([np.inf, -np.inf], np.nan, inplace=True)
df_tp.fillna(0, inplace=True)

df_train = df_tp[df_tp["commit_id"].isin(train_commit)]
df_train = df_train.reset_index().drop(columns=['index'])

df_test = df_tp[df_tp["commit_id"].isin(test_commit)]
df_test = df_test.reset_index().drop(columns=['index'])

df_train.columns = ['col_' + str(col).replace('-', '_') for col in df_train.columns]
df_test.columns = ['col_' + str(col).replace('-', '_') for col in df_test.columns]

print(df_train.shape, df_test.shape)

df_train["bugcount"] = train_label
df_test["bugcount"] = test_label

print(df_train.shape, df_test.shape)

(1709, 229) (734, 229)
(1709, 230) (734, 230)


In [10]:
df_test.iloc[1:2, 2:]

,col_asawr,col_oexp,col_nf,col_age,col_osawr,col_rtime,col_hcmt,col_arexp,col_ent,col_osexp,...,col_if_block_if_stmt_else_block_if_block_expr_call_name,col_if_block_if_stmt_else_block_if_block_expr_call_argument_expr_operator,col_for_block_if_block_decl_init_expr_call_name,col_decl_init_expr_ternary_else_expr_call_name_name,col_for_block_if_block_return_expr_operator,col_do_block_expr_name,col_if_block_if_block_decl_argument_expr_call_name,col_function_block_return_expr_ternary_condition_expr_call_name_name,col_decl_stmt_operator,bugcount
1,0.020938,646.0,1,1059535.0,0.028538,588151.0,3.0,380.0,0.0,566.0,...,0,0,0,0,0,0,0,0,0,1


In [11]:
y_train = df_train["bugcount"]
X_train = df_train.drop(['col_commit_id', 'bugcount'], axis=1)

y_test = df_test["bugcount"]
X_test = df_test.drop(['col_commit_id', 'bugcount'], axis=1)

rf_model = RandomForestClassifier(max_depth=100, random_state=42, n_estimators=100)


In [12]:
print(X_train.shape, y_train.shape)
#print(y_test.shape, list(y_test))
#print(list(y_preds.astype('int32')), "\n", list(y_test['bugcount'].astype('int32')))

(1709, 228) (1709,)


In [14]:
list_features = X_train.columns
best_features = 0
best_buggy = 0
best_nonbuggy=0
plot_x = []
plot_buggy_y = []
plot_nonbuggy_y = []

cur_features = 1
while cur_features < len(list_features):
    rf_model = RandomForestClassifier(random_state=42)
    rf_model.fit(X_train[list_features[0:cur_features]], y_train.values.ravel())

    y_preds = rf_model.predict(X_test[list_features[0:cur_features]]) 

    CorrectBuggyCommits, CorrectNonBuggyCommits = findCorrectBuggy(list(y_test.astype('int32')), 
                                                                   list(y_preds.astype('int32')))
    plot_x.append(cur_features)
    plot_buggy_y.append(len(CorrectBuggyCommits))
    plot_nonbuggy_y.append(len(CorrectNonBuggyCommits))
    
    if(best_buggy < len(CorrectBuggyCommits)):
        best_features = cur_features
        best_buggy = len(CorrectBuggyCommits)
        best_nonbuggy = len(CorrectNonBuggyCommits)
    elif(best_buggy == len(CorrectBuggyCommits) and best_nonbuggy < len(CorrectNonBuggyCommits)):
        best_features = cur_features
        best_nonbuggy = len(CorrectNonBuggyCommits)
        
    clear_output(wait=True)
    print("Working for: ", cur_features, "/", best_features, "/", best_buggy, "/", best_nonbuggy)
    cur_features += 1
    
#clear_output(wait=True)

Best Number of Features: 6
Number of Correct Buggy Commit Prediction:  151
Number of Correct Non-Buggy Commit Prediction:  187


In [45]:
#run again for the best values: 
X_train = X_train[list_features[0:best_features]]
X_test = X_test[list_features[0:best_features]] 
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train[list_features[0:best_features]], y_train.values.ravel())
y_preds = rf_model.predict(X_test) 

CorrectBuggyCommits, CorrectNonBuggyCommits = findCorrectBuggy(list(y_test.astype('int32')), 
                                                                   list(y_preds.astype('int32')))

print("Best Number of Features:", best_features)
print("Number of Correct Buggy Commit Prediction: ", len(CorrectBuggyCommits))
print("Number of Correct Non-Buggy Commit Prediction: ", len(CorrectNonBuggyCommits))

Best Number of Features: 6
Number of Correct Buggy Commit Prediction:  151
Number of Correct Non-Buggy Commit Prediction:  187


In [46]:
# create a DataFrame which only has predicted label column
y_preds = pd.DataFrame(data={'PredictedBug': y_preds}, index=df_test.index) 
#y_preds.head(3)

combined_testing_data = X_test.join(y_test.to_frame())
combined_testing_data = combined_testing_data.join(y_preds)
combined_testing_data.head(3)
# total num of rows
total_rows = len(combined_testing_data)

In [47]:
correctly_predicted_data = combined_testing_data[combined_testing_data['bugcount']==combined_testing_data['PredictedBug']]
correctly_predicted_rows = len(correctly_predicted_data)
print("Total testing data:", total_rows)
print('The model correctly predicted ', round((correctly_predicted_rows / total_rows), 3) * 100, '% of testing data')

Total testing data: 734
The model correctly predicted  46.0 % of testing data


In [48]:
print(Counter(correctly_predicted_data['bugcount']))

Counter({'0': 187, '1': 151})


In [49]:
#Takes only the correctly predicted buggy data
correctly_predicted_bug = correctly_predicted_data[correctly_predicted_data['bugcount']=='1']
print(correctly_predicted_bug.shape)
#correctly_predicted_bug.head(3)

(151, 8)


In [50]:
# select all rows and feature cols
feature_cols = correctly_predicted_bug.iloc[:, :-2]
# selected all rows and one label col (either RealBug or PredictedBug is fine since they are the same)
label_col = correctly_predicted_bug.iloc[:, -2]

In [43]:
pyexp = PyExplainer(X_train = X_train,
                    y_train = y_train,
                    indep = X_train.columns,
                    dep = 'bugcount',
                    blackbox_model = rf_model)

In [51]:
list_of_rules = []
selected_row = 0
while(selected_row < len(feature_cols)):
    clear_output(wait=True)
    print("Working for: ", selected_row, "/", len(feature_cols))
    # select the row in X_test which contains all of the feature values
    X_explain = feature_cols.iloc[[selected_row]]
    # select the corresponding label from the DataFrame that we just created above
    y_explain = label_col.iloc[[selected_row]]

    #this will reset the previous column to make it zero and drop the column created by reset function. 
    #It is important for .visualize function, bug is in the visualize function. 
    X_explain = X_explain.reset_index().drop(columns=['index'])
    #y_explain do not need it
    #y_explain = y_explain.reset_index().drop(columns=['index']).squeeze()

    created_rules = pyexp.explain(X_explain=X_explain,
                                  y_explain=y_explain,
                                  search_function='crossoverinterpolation',
                                  random_state=0,
                                  reuse_local_model=True)
    
    list_of_rules.append(created_rules)
    selected_row +=1

# Please use the code below to visualise the generated PyExplainer explanation (What-If interactive visualisation).
#pyexp.visualise(created_rules, title="Why this file is defect-introducing ?")


Working for:  150 / 151
Random Perturbation only generated one class for the prediction column which means 
                     Random Perturbation is not compatible with the current data. 
                     The 'Crossover and Interpolation' approach is used as the alternative.


In [52]:
print(len(list_of_rules))

151


In [53]:
rule_index = 4
pyexp.visualise(list_of_rules[rule_index], title="Why this file is defect-introducing ?")

Output(layout=Layout(border='3px solid black'))

FloatSlider(value=0.16, continuous_update=False, description='#1 The value of col_osawr is less than 0.16', la…

FloatSlider(value=3697.0, continuous_update=False, description='#2 The value of col_oexp is more than 3697.0',…

FloatSlider(value=110858.0, continuous_update=False, description='#3 The value of col_age is more than 110858.…

In [54]:
#list_of_rules[rule_index].keys()
print(list_of_rules[rule_index]['top_k_positive_rules']['rule'])

0      col_la <= 100.56999969482422 & col_nf > 1.5049...
1      col_asawr <= 0.014999999664723873 & col_la <= ...
2      col_age > 983335.15625 & col_age > 9714.649902...
3      col_la > 2.430000066757202 & col_asawr > 0.014...
4      col_la <= 14.335000038146973 & col_asawr > 0.0...
                             ...                        
141    col_osawr > 0.19500000029802322 & col_age <= 2...
142    col_nf > 4.144999980926514 & col_la > 75.88499...
143    col_la > 34.78500175476074 & col_nf <= 2.04499...
144    col_asawr > 0.014999999664723873 & col_la > 10...
145    col_age > 2616855.125 & col_la > 23.0450000762...
Name: rule, Length: 146, dtype: object


In [55]:
BuggyRuleDataset = [['commit_id', 'rule', 'counter' ]]
buggy_rule_features=[]
for rule_index in range(len(list_of_rules)):    
    single_rules = []
    for r in list_of_rules[rule_index]['top_k_positive_rules']['rule']:
        for sr in r.split('&'):
            single_rules.append(sr.strip())
            
    counted_rules = Counter(single_rules)
    for cr in counted_rules:
        BuggyRuleDataset.append([CorrectBuggyCommits[rule_index], cr.replace('col_', ''), counted_rules[cr]])
        if(cr.split()[0] not in buggy_rule_features):
            buggy_rule_features.append(cr.split()[0])

In [56]:
for br in BuggyRuleDataset:
    print(br)

['commit_id', 'rule', 'counter']
['adaa50e37da1fddada874a289792a1bd95c5aebc', 'nf <= 6.3500001430511475', 1]
['adaa50e37da1fddada874a289792a1bd95c5aebc', 'la <= 420.78001403808594', 1]
['adaa50e37da1fddada874a289792a1bd95c5aebc', 'la <= 44.545000076293945', 1]
['adaa50e37da1fddada874a289792a1bd95c5aebc', 'la > 3.4049999713897705', 1]
['adaa50e37da1fddada874a289792a1bd95c5aebc', 'nf > 2.259999990463257', 1]
['adaa50e37da1fddada874a289792a1bd95c5aebc', 'osawr <= 0.1249999962747097', 3]
['adaa50e37da1fddada874a289792a1bd95c5aebc', 'osawr > 0.11499999836087227', 5]
['adaa50e37da1fddada874a289792a1bd95c5aebc', 'la > 1.7800000309944153', 1]
['adaa50e37da1fddada874a289792a1bd95c5aebc', 'nf <= 2.390000104904175', 1]
['adaa50e37da1fddada874a289792a1bd95c5aebc', 'age > 829870.28125', 1]
['adaa50e37da1fddada874a289792a1bd95c5aebc', 'la <= 617.864990234375', 1]
['adaa50e37da1fddada874a289792a1bd95c5aebc', 'osawr <= 0.20499999821186066', 6]
['adaa50e37da1fddada874a289792a1bd95c5aebc', 'nf > 3.67000

['28ee554b37be39c03c231e7b857f71163dc6ea73', 'oexp > 2392.4349365234375', 1]
['28ee554b37be39c03c231e7b857f71163dc6ea73', 'oexp > 2545.1400146484375', 1]
['28ee554b37be39c03c231e7b857f71163dc6ea73', 'oexp <= 2625.330078125', 1]
['28ee554b37be39c03c231e7b857f71163dc6ea73', 'la <= 2.6399999856948853', 1]
['28ee554b37be39c03c231e7b857f71163dc6ea73', 'age > 793603.40625', 1]
['28ee554b37be39c03c231e7b857f71163dc6ea73', 'nf <= 2.2899999618530273', 1]
['28ee554b37be39c03c231e7b857f71163dc6ea73', 'la > 104.5', 1]
['28ee554b37be39c03c231e7b857f71163dc6ea73', 'oexp <= 2087.8349609375', 1]
['28ee554b37be39c03c231e7b857f71163dc6ea73', 'age > 787595.0625', 1]
['28ee554b37be39c03c231e7b857f71163dc6ea73', 'oexp <= 2678.294921875', 1]
['28ee554b37be39c03c231e7b857f71163dc6ea73', 'age > 1050037.375', 1]
['28ee554b37be39c03c231e7b857f71163dc6ea73', 'osawr <= 0.0950000025331974', 1]
['28ee554b37be39c03c231e7b857f71163dc6ea73', 'oexp <= 2623.6900634765625', 1]
['28ee554b37be39c03c231e7b857f71163dc6ea73',

['3168c94577cc604fd033ce4e741da5c411f74666', 'la <= 23.045000076293945', 1]
['3168c94577cc604fd033ce4e741da5c411f74666', 'nf <= -2.1549999713897705', 1]
['3168c94577cc604fd033ce4e741da5c411f74666', 'osawr <= 0.06499999947845936', 1]
['3168c94577cc604fd033ce4e741da5c411f74666', 'la <= 3.8399999141693115', 1]
['3168c94577cc604fd033ce4e741da5c411f74666', 'nf <= 4.994999885559082', 1]
['3168c94577cc604fd033ce4e741da5c411f74666', 'oexp <= 1544.280029296875', 1]
['3168c94577cc604fd033ce4e741da5c411f74666', 'age > 692060.46875', 1]
['3168c94577cc604fd033ce4e741da5c411f74666', 'oexp > 2724.455078125', 1]
['3168c94577cc604fd033ce4e741da5c411f74666', 'oexp <= 1623.6649780273438', 1]
['3168c94577cc604fd033ce4e741da5c411f74666', 'osawr > 0.07499999925494194', 1]
['3168c94577cc604fd033ce4e741da5c411f74666', 'asawr <= -0.029999999329447746', 1]
['3168c94577cc604fd033ce4e741da5c411f74666', 'la > 1.074999988079071', 1]
['3168c94577cc604fd033ce4e741da5c411f74666', 'age > 3626620.25', 1]
['3168c94577cc6

['51980595e100a1f124e77f35ddfdd5cfd2f0722b', 'oexp <= 1521.7449951171875', 1]
['51980595e100a1f124e77f35ddfdd5cfd2f0722b', 'oexp <= 1544.280029296875', 1]
['51980595e100a1f124e77f35ddfdd5cfd2f0722b', 'age > 692060.46875', 1]
['51980595e100a1f124e77f35ddfdd5cfd2f0722b', 'oexp > 2724.455078125', 1]
['51980595e100a1f124e77f35ddfdd5cfd2f0722b', 'oexp <= 1623.6649780273438', 1]
['51980595e100a1f124e77f35ddfdd5cfd2f0722b', 'osawr > 0.07499999925494194', 1]
['51980595e100a1f124e77f35ddfdd5cfd2f0722b', 'asawr <= -0.029999999329447746', 1]
['51980595e100a1f124e77f35ddfdd5cfd2f0722b', 'la > 118.6049976348877', 1]
['51980595e100a1f124e77f35ddfdd5cfd2f0722b', 'oexp > 2544.3599853515625', 1]
['51980595e100a1f124e77f35ddfdd5cfd2f0722b', 'age > 3626620.25', 1]
['51980595e100a1f124e77f35ddfdd5cfd2f0722b', 'oexp <= 1608.4949951171875', 1]
['51980595e100a1f124e77f35ddfdd5cfd2f0722b', 'la > 34.07499885559082', 1]
['51980595e100a1f124e77f35ddfdd5cfd2f0722b', 'nf <= 2.840000033378601', 1]
['51980595e100a1f

['6281acaf8f5cd11a7fa6eb40da68b4707a41e148', 'la <= 132.8350067138672', 1]
['6281acaf8f5cd11a7fa6eb40da68b4707a41e148', 'age <= 2355888.5', 1]
['6281acaf8f5cd11a7fa6eb40da68b4707a41e148', 'age > 635392.15625', 1]
['6281acaf8f5cd11a7fa6eb40da68b4707a41e148', 'nf > 0.2900000065565109', 1]
['6281acaf8f5cd11a7fa6eb40da68b4707a41e148', 'nf <= 6.359999895095825', 1]
['6281acaf8f5cd11a7fa6eb40da68b4707a41e148', 'la <= 422.0350036621094', 1]
['6281acaf8f5cd11a7fa6eb40da68b4707a41e148', 'oexp > 2546.60498046875', 1]
['6281acaf8f5cd11a7fa6eb40da68b4707a41e148', 'age <= 1880680.1875', 1]
['6281acaf8f5cd11a7fa6eb40da68b4707a41e148', 'asawr > 0.08500000089406967', 2]
['6281acaf8f5cd11a7fa6eb40da68b4707a41e148', 'la <= 73.22999954223633', 1]
['6281acaf8f5cd11a7fa6eb40da68b4707a41e148', 'nf > 2.0449999570846558', 1]
['6281acaf8f5cd11a7fa6eb40da68b4707a41e148', 'age > 324318.171875', 1]
['6281acaf8f5cd11a7fa6eb40da68b4707a41e148', 'la <= 3.2450000047683716', 1]
['6281acaf8f5cd11a7fa6eb40da68b4707a41e1

...

['8e2f95e5a2ebc54c600d1db416b1dd84e433d115', 'nf > 5.855000019073486', 1]
['8e2f95e5a2ebc54c600d1db416b1dd84e433d115', 'la > 270.4049987792969', 1]
['8e2f95e5a2ebc54c600d1db416b1dd84e433d115', 'la <= 374.0850067138672', 1]
['8e2f95e5a2ebc54c600d1db416b1dd84e433d115', 'age > 831732.875', 1]
['8e2f95e5a2ebc54c600d1db416b1dd84e433d115', 'la <= 621.2950134277344', 2]
['8e2f95e5a2ebc54c600d1db416b1dd84e433d115', 'oexp > 2143.9649658203125', 1]
['8e2f95e5a2ebc54c600d1db416b1dd84e433d115', 'oexp > 1893.2150268554688', 1]
['8e2f95e5a2ebc54c600d1db416b1dd84e433d115', 'age > 3001247.75', 1]
['8e2f95e5a2ebc54c600d1db416b1dd84e433d115', 'oexp <= 2702.719970703125', 1]
['8e2f95e5a2ebc54c600d1db416b1dd84e433d115', 'oexp > 2382.6700439453125', 1]
['8e2f95e5a2ebc54c600d1db416b1dd84e433d115', 'nf > -2.8200000524520874', 1]
['8e2f95e5a2ebc54c600d1db416b1dd84e433d115', 'age > 1879669.0625', 1]
['8e2f95e5a2ebc54c600d1db416b1dd84e433d115', 'la <= 2.5649999380111694', 1]
['8e2f95e5a2ebc54c600d1db416b1dd84e4

['b79e73476771c068098270ebc26fb1e015f0e149', 'age > 324318.171875', 1]
['b79e73476771c068098270ebc26fb1e015f0e149', 'la <= 3.2450000047683716', 2]
['b79e73476771c068098270ebc26fb1e015f0e149', 'age <= 269934.875', 1]
['b79e73476771c068098270ebc26fb1e015f0e149', 'oexp > 2670.794921875', 1]
['b79e73476771c068098270ebc26fb1e015f0e149', 'age > 3468956.25', 1]
['b79e73476771c068098270ebc26fb1e015f0e149', 'oexp <= 2004.5700073242188', 1]
['b79e73476771c068098270ebc26fb1e015f0e149', 'age <= 2392887.125', 1]
['b79e73476771c068098270ebc26fb1e015f0e149', 'age <= 1002726.65625', 1]
['b79e73476771c068098270ebc26fb1e015f0e149', 'age > 772538.8125', 1]
['b79e73476771c068098270ebc26fb1e015f0e149', 'la <= 3.090000033378601', 1]
['b79e73476771c068098270ebc26fb1e015f0e149', 'nf > 1.9950000047683716', 1]
['b79e73476771c068098270ebc26fb1e015f0e149', 'la > -2.3049999475479126', 1]
['b79e73476771c068098270ebc26fb1e015f0e149', 'oexp <= 1659.7300415039062', 1]
['b79e73476771c068098270ebc26fb1e015f0e149', 'age 

['b3aed29c434994f2e980a8ca371ab83b48e5d98b', 'age > 9714.64990234375', 1]
['b3aed29c434994f2e980a8ca371ab83b48e5d98b', 'oexp > 1316.8150024414062', 1]
['b3aed29c434994f2e980a8ca371ab83b48e5d98b', 'la <= 14.335000038146973', 1]
['b3aed29c434994f2e980a8ca371ab83b48e5d98b', 'asawr > 0.004999999888241291', 6]
['b3aed29c434994f2e980a8ca371ab83b48e5d98b', 'nf <= 1.034999966621399', 1]
['b3aed29c434994f2e980a8ca371ab83b48e5d98b', 'osawr <= 0.19500000029802322', 2]
['b3aed29c434994f2e980a8ca371ab83b48e5d98b', 'la > 3.5049999952316284', 1]
['b3aed29c434994f2e980a8ca371ab83b48e5d98b', 'age <= 764118.75', 1]
['b3aed29c434994f2e980a8ca371ab83b48e5d98b', 'osawr > 0.11499999836087227', 6]
['b3aed29c434994f2e980a8ca371ab83b48e5d98b', 'la > 2.5299999713897705', 1]
['b3aed29c434994f2e980a8ca371ab83b48e5d98b', 'age > 430110.03125', 1]
['b3aed29c434994f2e980a8ca371ab83b48e5d98b', 'osawr <= 0.11499999836087227', 8]
['b3aed29c434994f2e980a8ca371ab83b48e5d98b', 'osawr <= 0.13499999791383743', 1]
['b3aed29c4

['b3f201b8afcf88015f5bff7b3f43e90b13f8a8f1', 'nf <= 6.494999885559082', 1]
['b3f201b8afcf88015f5bff7b3f43e90b13f8a8f1', 'nf > 5.855000019073486', 1]
['b3f201b8afcf88015f5bff7b3f43e90b13f8a8f1', 'la <= 621.2950134277344', 2]
['b3f201b8afcf88015f5bff7b3f43e90b13f8a8f1', 'la > -188.63500213623047', 1]
['b3f201b8afcf88015f5bff7b3f43e90b13f8a8f1', 'age > 2397230.875', 1]
['b3f201b8afcf88015f5bff7b3f43e90b13f8a8f1', 'oexp > 2134.9949951171875', 1]
['b3f201b8afcf88015f5bff7b3f43e90b13f8a8f1', 'age > 3001247.75', 1]
['b3f201b8afcf88015f5bff7b3f43e90b13f8a8f1', 'asawr > 0.024999999441206455', 3]
['b3f201b8afcf88015f5bff7b3f43e90b13f8a8f1', 'oexp <= 2651.364990234375', 1]
['b3f201b8afcf88015f5bff7b3f43e90b13f8a8f1', 'oexp > 2382.6700439453125', 1]
['b3f201b8afcf88015f5bff7b3f43e90b13f8a8f1', 'nf > -2.8200000524520874', 1]
['b3f201b8afcf88015f5bff7b3f43e90b13f8a8f1', 'age > 1879669.0625', 1]
['b3f201b8afcf88015f5bff7b3f43e90b13f8a8f1', 'nf <= 2.284999966621399', 1]
['b3f201b8afcf88015f5bff7b3f43e

['90808ead98edbab33a6bcc724d123864842a5ed3', 'oexp <= 2757.5650634765625', 1]
['90808ead98edbab33a6bcc724d123864842a5ed3', 'la > -79.6150016784668', 1]
['90808ead98edbab33a6bcc724d123864842a5ed3', 'oexp > 1528.2900390625', 1]
['90808ead98edbab33a6bcc724d123864842a5ed3', 'age <= 3051830.375', 1]
['90808ead98edbab33a6bcc724d123864842a5ed3', 'age > 840367.5625', 1]
['90808ead98edbab33a6bcc724d123864842a5ed3', 'la <= 615.7999877929688', 1]
['90808ead98edbab33a6bcc724d123864842a5ed3', 'age > 3474925.375', 1]
['90808ead98edbab33a6bcc724d123864842a5ed3', 'oexp <= 2545.1400146484375', 2]
['90808ead98edbab33a6bcc724d123864842a5ed3', 'oexp > 1544.280029296875', 1]
['90808ead98edbab33a6bcc724d123864842a5ed3', 'la <= 132.8350067138672', 1]
['90808ead98edbab33a6bcc724d123864842a5ed3', 'age <= 2355888.5', 1]
['90808ead98edbab33a6bcc724d123864842a5ed3', 'age > 635392.15625', 1]
['90808ead98edbab33a6bcc724d123864842a5ed3', 'oexp > 2545.1400146484375', 1]
['90808ead98edbab33a6bcc724d123864842a5ed3', 'n

In [57]:
print(len(buggy_rule_features))
for brf in buggy_rule_features:
    print(brf.replace('col_', '').replace('_', '-'))

6
nf
la
osawr
age
oexp
asawr


In [73]:
rule_report = []
list_of_rules = []
for br in BuggyRuleDataset:
    list_of_rules.append(br[1])

counted_rule = Counter(list_of_rules)
for cr in counted_rule:
    rule_report.append([cr, counted_rule[cr]])

In [76]:
sorted_rules = pd.DataFrame(rule_report, columns=['rule', 'count']).sort_values(by=['count'], ascending=False)

In [77]:
final_text = "Count, Rule\n"
for i in sorted_rules.index:
    final_text += str(sorted_rules['count'][i])+": "+sorted_rules['rule'][i]+'\n'
    
print(final_text)

Count, Rule
151: asawr <= 0.024999999441206455
151: asawr > 0.004999999888241291
151: osawr <= 0.11499999836087227
151: osawr > 0.11499999836087227
151: asawr <= 0.03499999921768904
151: osawr <= 0.0950000025331974
151: asawr <= 0.014999999664723873
151: asawr > 0.014999999664723873
150: osawr <= 0.19500000029802322
150: osawr <= 0.20499999821186066
148: age <= 647989.4375
148: oexp > 2721.8800048828125
147: osawr <= 0.17500000447034836
145: age > 831732.875
145: osawr > 0.1550000011920929
141: age > 694101.53125
138: asawr > 0.03499999921768904
137: age > 3626620.25
136: asawr > 0.07499999925494194
136: osawr <= 0.1850000023841858
134: osawr > 0.1249999962747097
134: la <= 621.2950134277344
134: osawr <= 0.1249999962747097
133: osawr > 0.14500000327825546
132: nf <= 2.259999990463257
132: la > 57.59499931335449
132: osawr <= 0.13499999791383743
131: age > 635392.15625
130: asawr <= 0.004999999888241291
126: osawr > 0.13499999791383743
125: asawr <= 0.08500000089406967
124: age > 12208